# 程序 `sdf_reader`

In [1]:
import basis_set_exchange as bse
import numpy as np

from sdf_reader import *

RDKit WARNING: [19:11:24] Enabling RDKit 2019.09.1 jupyter extensions


## 字典 `slater_zeta_valance`

譬如对于氯原子，字典值为

In [2]:
slater_zeta_valance["Cl"]

[[16.43, 6.26, 6.26, 6.26, 6.26, 2.1, 2.1, 2.1, 2.1],
 [0, 0, 0, 0, 0, 1, 1, 1, 1]]

第一行是 STO-3G 基组的指数系数 $\zeta$，第二行则标明是否是价层轨道。STO-3G 的基组指数系数 $\zeta$ 可以在 Szabo, p186, Table 3.8 中找到其中的一部分。对于 Cl 原子而言，其第一层轨道的指数系数 $\zeta$ 可以用如下的方式给出：

In [3]:
Cl_1s_expo = np.array(bse.get_basis('STO-3G', elements=["Cl"])["elements"]["17"]["electron_shells"][0]["exponents"], dtype=float)
Cl_1s_expo

array([601.3456136 , 109.5358542 ,  29.64467686])

In [4]:
H_1s_expo = np.array(bse.get_basis('STO-3G', elements=["H"])["elements"]["1"]["electron_shells"][0]["exponents"], dtype=float)
H_1s_expo

array([3.42525091, 0.62391373, 0.1688554 ])

In [5]:
np.sqrt(Cl_1s_expo / H_1s_expo) * 1.24

array([16.43, 16.43, 16.43])

因此，Cl 原子的 1s 基组轨道的指数系数 $\zeta$ 是 16.43。第二层、第三层 (价层) 等同理。

## 函数 `sdf_to_dict`

我们拿分子 `1409364` 来解释这个函数的意义。

In [6]:
dict_1409364 = sdf_to_dict("raw-sdf/dev/sdf/atom_11/1409364.sdf")

通过阅读 Alchemy 数据集的原始 .sdf 文件，返回具有物理或化学意义的各种矩阵。`dict_1409364` 本身是长度为 2 的 tuple，其中前一个元素是原子特征字典，后一个元素是轨道特征字典。

In [7]:
for entry in dict_1409364[0]:
    print("{:20} shape {:20}".format(entry, str(dict_1409364[0][entry].shape)))

atm_coord            shape (17, 3)             
atm_dist             shape (17, 17)            
atm_charge           shape (17,)               
atm_nuceng_adaj      shape (17, 17)            
atm_symbol_onehot    shape (17, 7)             
atm_addcharge        shape (17,)               
atm_aromatic         shape (17,)               
atm_hybrid           shape (17, 3)             
atm_edge_type        shape (5, 17, 17)         


其中，

| 字典索引 | 意义 |
|:--------:|:----:|
| `atm_coord` | 原子坐标 |
| `atm_dist` | 原子间距离 |
| `atm_charge` | 原子电荷 |
| `atm_nuceng_adaj` | 核排斥能矩阵 |
| `atm_symbol_onehot` | 原子名称的 one-hot 向量 |
| `atm_addcharge` | 原子带电荷 (经验量) |
| `atm_aromatic` | 原子是否是芳香性的 (经验量) |
| `atm_hybrid` | 原子的杂化形式 (sp, sp2, sp3) (经验量) |
| `atm_edge_type` | 键的种类 (单键、双键、叁键、芳香、未知) (经验量) |

In [8]:
for entry in dict_1409364[1]:
    try:
        print("{:20} shape {:20}".format(entry, str(dict_1409364[1][entry].shape)))
    except:
        print("{:20} value  {:20}".format(entry, str(dict_1409364[1][entry])))

nelec                value  86                  
int1e_ovlp           shape (65, 65)            
int1e_kin            shape (65, 65)            
int1e_nuc            shape (65, 65)            
int1e_r              shape (3, 65, 65)         
rdm1e                shape (65, 65)            
ao_idx               shape (65,)               
ao_atomchg           shape (65,)               
ao_atomhot           shape (65, 7)             
ao_zeta              shape (65,)               
ao_valence           shape (65,)               
ao_momentum          shape (65,)               
ao_spacial_x         shape (65,)               
ao_spacial_y         shape (65,)               
ao_spacial_z         shape (65,)               
ao_coord             shape (65, 3)             
ao_dist              shape (65, 65)            


| 字典索引 | 意义 |
|:--------:|:----:|
| `nelec` | 电子数 |
| `int1e_ovlp` | 重叠矩阵 |
| `int1e_kin` | 动能积分矩阵 |
| `int1e_nuc` | 核势能积分矩阵 |
| `int1e_r` | 偶极积分矩阵 |
| `rdm1e` | 密度初猜 (通过 $F=T+V$ 构造) |
| `ao_idx` | 轨道对应的原子的号码 |
| `ao_atomchg` | 轨道对应原子的核电荷 |
| `ao_atomhot` | 轨道对应原子的 one-hot 编码 |
| `ao_zeta` | 轨道的指数系数 $\zeta$ |
| `ao_valence` | 轨道是否是价层 |
| `ao_momentum` | 轨道是 $s$ 或是 $p$ |
| `ao_spacial_x` | 若为 $p$ 轨道，是否为 $x$ 方向取向 |
| `ao_spacial_y` | 若为 $p$ 轨道，是否为 $y$ 方向取向 |
| `ao_spacial_z` | 若为 $p$ 轨道，是否为 $z$ 方向取向 |
| `ao_coord` | 轨道中心的坐标 (对应原子的坐标) |
| `ao_dist` | 轨道中心坐标间的距离 |

## `sdf_reader` 主程序

处理所有原始的 .sdf 文件，将处理得到的特征储存到 .dat 文件。

这一个过程需要用到量化软件，相对耗时；但由于不经过 SCF 过程，因此耗时不会太大。最大的计算量是 `int1e_nuc`，计算复杂度 $O(n_\mathrm{atom} n_{AO}^2)$。

但处理 10 万分子仍然会是一个不小的工作量。这一个程序的主要目的是将特征提取出来；以后的程序就不必在特征提取的过程中耗费时间。